https://dse-souken.com/2021/05/18/ai-17/
スクリプトは上記から引用した。

In [2]:
#必要なモジュールをインポート
import numpy as np

gamma = 0.9#将来価値の割引。小さいほど行動直後の利益を重視。また、この割引率の存在が効率的な学習の鍵となる。
alpha = 0.1#学習率。大きいと1回の学習による値の更新が急激となる。小さいほど更新がゆるやかとなる。後で詳述。

#報酬の設定
#各場所から移動できる箇所に報酬1を与え、それ以外を0とすることで移動できる方向を指示
#以下の行列の各行が場所に対応。0行目は迷路の位置0、1行目が迷路の位置1
#ゴールとなる8の報酬を大きく設定する
reward = np.array([[0,1,0,1,0,0,0,0,0],#0から1,3に移動できるので、そこに1
                   [1,0,1,0,1,0,0,0,0],#1から0,2,4に移動できるので、そこに1
                   [0,1,0,0,0,0,0,0,0],
                   [1,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,1,0,1,0],
                   [0,0,0,0,1,0,0,0,10000],#8をゴールより5→8の報酬を大きく
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,0],
                   [0,0,0,0,0,1,0,0,0]])

#Q値(行動価値)の初期値を設定。今回は0を初期値とする。
Q = np.array(np.zeros([9,9]))

#Q学習を実装し、各位置における行動価値を算出
#以下の学習を実行すると、行動価値Qを求められる。Qの各行が位置に対応し、たとえば0行1列目の値は0から1に移動する行動の価値となる。
#p_stateのpはpresent(現在)、n_state,n_actionsのnはnext(次)のn
for i in range(10000):#1万回繰り返し学習を行う
    p_state = np.random.randint(0,9)#現在の状態をランダムに選択
    n_actions = []#次の行動の候補を入れる箱
    for j in range(9):
        if reward[p_state,j] >= 1:#rewardの各行が1以上のインデックスを取得
            n_actions.append(j)#これでp_stateの状態で移動できる場所を取得
    n_state = np.random.choice(n_actions)#行動可能選択肢からランダムに選択
    
    #Q値の更新。学習率が小さいほど現在の行動価値が重視され、更新がゆるやかとなる
    #ここでQ学習に用いる「たった一つの数式」を利用して行動価値を学習していく
    Q[p_state,n_state] = (1-alpha)*Q[p_state,n_state]+alpha*(reward[p_state,n_state]+gamma*Q[n_state,np.argmax(Q[n_state,])])

#最短ルート表示関数の定義。Q値が最も高い行動をappendで追加しているだけ
def shortest_path(start):#0～8の数字を入力。好きなところからスタート可能
    path = [start]#pathに経路を追加していく
    p_pos = start#p_posは現在位置(positionの略)
    n_pos = p_pos#n_pos(次の位置）にいったんp_posを代入
    while(n_pos != 8):#n_posがゴール（8）になるまで繰り返し行動を選択
        n_pos = np.argmax(Q[p_pos,])#各位置の行動価値が最も高い行動を選択
        path.append(n_pos)#経路をpathに追加
        p_pos = n_pos#行動後が次のp_posとなる
    return path

print(shortest_path(0))#スタートを0として最短経路を表示
print(Q)

[0, 1, 4, 5, 8]
[[    0.         38337.29360732     0.         31048.8163105
      0.             0.             0.             0.
      0.        ]
 [34497.50165261     0.         34498.57122732     0.
  42598.01264573     0.             0.             0.
      0.        ]
 [    0.         38338.35793626     0.             0.
      0.             0.             0.             0.
      0.        ]
 [34501.13267039     0.             0.             0.
      0.             0.         34501.46193262     0.
      0.        ]
 [    0.         38330.40723466     0.             0.
      0.         47339.59937017     0.         38336.45869581
      0.        ]
 [    0.             0.             0.             0.
  42603.74023701     0.             0.             0.
  52603.02024269]
 [    0.             0.             0.         31047.22739434
      0.             0.             0.         38336.98696443
      0.        ]
 [    0.             0.             0.             0.
  42599.45504966 

上記Qを見ると、最短経路で進むようにQが学習されている。
例えば、1個目の要素を見ると
[    0.         38337.29360732     0.         31048.8163105
      0.             0.             0.             0.
      0.        ]
となっている。1個目の要素の時点では、エージェントは0の位置におり、1のアクションを取ることで38337、3のアクションを取ることで31048、それ以外のアクションを取ることで0の報酬が得られることがわかる。
このとき、エージェントは最大の報酬を得るために動くから1の方向に進む。
これを繰り返すことで最短経路を発見することができる。